In [1]:
import json
import csv
import pandas as pd
import gzip
import glob
import os

main_cols = ['input', 'measurement_start_time', 'probe_asn', 'probe_cc', 'probe_ip','probe_network_name','resolver_asn', "report_id","resolver_asn",'resolver_ip', 'resolver_network_name', "solftware_name",
'test_name', 'test_runtime', 'test_start_time']

special = {'test_keys':["dns_experiment_failure","dns_consistency","control_failure","http_experiment_failure","body_length_match","body_proportion","status_code_match","headers_match","title_match","accessible","blocking","x_status"]}

special_special = {'test_keys':{"queries":{"answers":["asn","as_org_name", "ipv4"]}}}

df_columns = main_cols.copy()
df_columns = df_columns+special['test_keys']
df_columns = df_columns+ ["test_keys_" + i for i in special_special['test_keys']["queries"]["answers"]]
# df = pd.DataFrame(columns=df_columns)


In [11]:
def generate_dates(start_date, end_date):
    lst =  pd.date_range(start_date, end_date, freq='D')
    
    list_date = []
    for i in range(len(lst)):
        list_date.append(lst[i].date().strftime("%Y-%m-%d"))
    return list_date
dates = generate_dates('2021-06-20','2022-07-20')
def convert_to_csv(filename):
    with gzip.open(filename, 'rb') as f:
        file_content = f.readlines() 
    thelist =[]
    for i in range(len(file_content)):
        index = 0
        lst =["" for i in range(len(df_columns))]
        a = file_content[i].decode('UTF-8')
        dic = json.loads(a)

        for col in main_cols:
            if col in dic.keys():
                lst[index] = dic[col]
            index+=1
        test_keys = dic["test_keys"]
        for col in special["test_keys"]:
            if col in test_keys.keys():
                lst[index] = test_keys[col]
            index+=1
            
        answers = dic["test_keys"]["queries"]#[0]["answers"][0]
        if dic["test_keys"]["queries"] is not None:
            for j in range(len(dic["test_keys"]["queries"])):
                answer = dic["test_keys"]["queries"][j]["answers"]
                if answer is not None:
                    for t in range(len(answer)):
                        sub_index=index


                        ans = answer[t]

                        for col in special_special["test_keys"]["queries"]["answers"]:
                            if col in ans.keys():
                                lst[sub_index] = str(ans[col])+ "_"
                            sub_index+=1

        thelist.append(lst)
    df = pd.DataFrame(data = thelist, columns = df_columns)
    
    ### columns need to be processed again
    cols_need_processing = ['test_keys_asn', 'test_keys_as_org_name',
       'test_keys_ipv4']
    for col in cols_need_processing:
        list_ = list(df[col])
        list_=[i[:-1].split("_") for i in list_]
        df[col]=list_

    
    
    folder = "/".join(filename.split("/")[:-1])
    new_filename = filename.split("/")[-1].split(".")[0]
#     print(df)
#     df.to_csv(folder+"/"+new_filename+".csv")
    return df

In [12]:
def combine_data_by_day(day, country):
    ls = []
    for filename in glob.glob("/data/censorship/OONI/"+day+"/*/"+country+"/*.jsonl.gz"):
        df = convert_to_csv(filename)
        ls.append(df)
    if len(ls)>0:
        df = pd.concat(ls)
        print(df.shape)
        if not os.path.isdir("/data/censorship/OONI/"+day+"/"+country):
            cmd = "mkdir {0};".format("/data/censorship/OONI/"+day+"/"+country)
            os.system(cmd)

        df.to_csv("/data/censorship/OONI/"+day+"/"+country+"/combined.csv")
        
        

In [ ]:
for date in dates:
    print(date)
    combine_data_by_day(date, "CN")

2021-06-20
(3769, 30)
2021-06-21
(5248, 30)
2021-06-22
(4046, 30)
2021-06-23
(4170, 30)
2021-06-24


In [11]:
# filename = "/data/censorship/OONI/2022-01-29/07/TM/2022012907_TM_webconnectivity.n0.0.jsonl.gz"
# with gzip.open(filename, 'rb') as f:
#     file_content = f.readlines() 
# thelist =[]
# for i in range(len(file_content)):
#     index = 0
#     lst =["" for i in range(len(df_columns))]
#     a = file_content[i].decode('UTF-8')
#     dic = json.loads(a)
#     thelist.append(dic)
# thelist[0]

{'annotations': {'architecture': 'arm',
  'engine_name': 'ooniprobe-engine',
  'engine_version': '3.13.0',
  'flavor': 'stableFull',
  'network_type': 'wifi',
  'platform': 'android'},
 'data_format_version': '0.2.0',
 'input': 'https://www.tiktok.com/',
 'measurement_start_time': '2022-01-29 07:37:54',
 'probe_asn': 'AS20661',
 'probe_cc': 'TM',
 'probe_ip': '127.0.0.1',
 'probe_network_name': 'State Company of Electro Communications Turkmentelecom',
 'report_id': '20220129T073752Z_webconnectivity_TM_20661_n1_yzuXxqAMoqW9rjSI',
 'resolver_asn': 'AS13335',
 'resolver_ip': '172.68.14.207',
 'resolver_network_name': 'Cloudflare, Inc.',
 'software_name': 'ooniprobe-android',
 'software_version': '3.5.0',
 'test_helpers': {'backend': {'address': 'https://wcth.ooni.io',
   'type': 'https'}},
 'test_keys': {'agent': 'redirect',
  'client_resolver': '172.68.14.207',
  'retries': None,
  'socksproxy': None,
  'network_events': [{'address': '127.0.0.1:443',
    'failure': 'connection_refused',


In [10]:
ls = []
for date in dates:
    filename = "/data/censorship/OONI/"+date+"/"+"CN"+"/combined.csv"
    if os.path.exists(filename):
        df = pd.read_csv(filename)
        ls.append(df)
final = pd.concat(ls)
final
# final.to_csv("/data/censorship/OONI/TM_2021_06_20_to_2022_02_09.csv")

        
    

,Unnamed: 0,input,measurement_start_time,probe_asn,probe_cc,probe_ip,probe_network_name,resolver_asn,report_id,resolver_asn.1,...,body_proportion,status_code_match,headers_match,title_match,accessible,blocking,x_status,test_keys_asn,test_keys_as_org_name,test_keys_ipv4
0,0,https://www.tahr.org.tw/,2021-06-20 06:02:42,AS56046,CN,127.0.0.1,<unknown>,AS56046,20210620T042508Z_webconnectivity_CN_56046_n1_W...,AS56046,...,0.0,NaN,NaN,NaN,False,dns,4256,['13414'],['Twitter Inc.'],['199.16.156.40']
1,1,https://bridges.torproject.org/,2021-06-20 06:03:20,AS56046,CN,127.0.0.1,<unknown>,AS56046,20210620T042508Z_webconnectivity_CN_56046_n1_W...,AS56046,...,0.0,NaN,NaN,NaN,False,dns,4256,['13414'],['Twitter Inc.'],['185.45.7.185']
2,2,http://www.jmarshall.com/tools/cgiproxy/,2021-06-20 06:53:44,AS4134,CN,127.0.0.1,China Telecom,AS4134,20210620T065344Z_webconnectivity_CN_4134_n1_eb...,AS4134,...,1.0,True,False,True,True,False,1,['47172'],['Greenhost BV'],['213.108.104.108']
3,3,https://openvpn.net/,2021-06-20 06:53:48,AS4134,CN,127.0.0.1,China Telecom,AS4134,20210620T065344Z_webconnectivity_CN_4134_n1_eb...,AS4134,...,0.0,True,True,NaN,True,False,1,['13335'],"['Cloudflare, Inc.']",['104.18.110.96']
4,4,http://www.almanar.com.lb/,2021-06-20 06:53:51,AS4134,CN,127.0.0.1,China Telecom,AS4134,20210620T065344Z_webconnectivity_CN_4134_n1_eb...,AS4134,...,0.0,True,False,NaN,True,False,1,['20860'],['IOMART CLOUD SERVICES LIMITED'],['77.246.170.51']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4165,70,https://cn.nytimes.com/,2021-06-23 01:57:28,AS56046,CN,127.0.0.1,<unknown>,AS56046,20210622T145635Z_webconnectivity_CN_56046_n1_b...,AS56046,...,0.0,NaN,NaN,NaN,False,dns,8736,['32934'],"['Facebook, Inc.']",['31.13.65.1']
4166,71,https://1.0.0.3/dns-query?dns=q80BAAABAAAAAAAA...,2021-06-23 01:57:26,AS56046,CN,127.0.0.1,<unknown>,AS56046,20210623T015526Z_webconnectivity_CN_56046_n1_2...,AS56046,...,0.0,NaN,NaN,NaN,False,tcp_ip,4224,['13335'],"['Cloudflare, Inc.']",['1.0.0.3']
4167,72,http://technorati.com/,2021-06-23 01:57:57,AS56046,CN,127.0.0.1,<unknown>,AS56046,20210623T015526Z_webconnectivity_CN_56046_n1_2...,AS56046,...,1.0,True,False,True,True,False,1,['36271'],"['Synacor, Inc.']",['64.8.70.102']
4168,73,http://multiproxy.org/,2021-06-23 01:58:04,AS56046,CN,127.0.0.1,<unknown>,AS56046,20210623T015526Z_webconnectivity_CN_56046_n1_2...,AS56046,...,0.0,NaN,NaN,NaN,NaN,NaN,8192,['13335'],"['Cloudflare, Inc.']",['104.21.32.230']


In [39]:
filename = "/data/censorship/OONI/TM_2021_06_20_to_2022_02_09.csv"
df = pd.read_csv(filename)
print(df.columns)
columns_2_keep = ['input','measurement_start_time','probe_asn', 'probe_cc', 'probe_ip','resolver_asn','test_keys_ipv4']
df_ = df[columns_2_keep]
dns_blocking_truth = []
for ip in df_["test_keys_ipv4"]:
    ip=ip[2:-2]
    print(ip)
    if ip=="127.0.0.1":
        dns_blocking_truth.append("Confirmed")
    else:
        dns_blocking_truth.append(None)
df_["dns_blocking_truth"]=dns_blocking_truth
df_.to_csv("/data/censorship/OONI/TM_2021_06_20_to_2022_02_09groundtruth_combined.csv")
        

Index(['Unnamed: 0', 'Unnamed: 0.1', 'input', 'measurement_start_time',
       'probe_asn', 'probe_cc', 'probe_ip', 'probe_network_name',
       'resolver_asn', 'report_id', 'resolver_asn.1', 'resolver_ip',
       'resolver_network_name', 'solftware_name', 'test_name', 'test_runtime',
       'test_start_time', 'dns_experiment_failure', 'dns_consistency',
       'control_failure', 'http_experiment_failure', 'body_length_match',
       'body_proportion', 'status_code_match', 'headers_match', 'title_match',
       'accessible', 'blocking', 'x_status', 'test_keys_asn',
       'test_keys_as_org_name', 'test_keys_ipv4'],
      dtype='object')


151.101.194.49
2.21.199.243
104.198.14.52

34.199.236.120
167.172.21.207
35.162.48.252




151.101.2.29
184.154.65.252
204.44.192.39
158.232.15.93
23.185.0.2


72.47.224.138
188.165.51.93
34.149.2.250

104.16.104.144
51.15.185.86

193.138.233.46
81.19.78.83

208.94.3.17
178.248.234.223

188.93.21.98
81.52.143.38
91.239.231.6
128.6.43.23
141.193.213.20

/tmp/ipykernel_2748368/932320375.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_["dns_blocking_truth"]=dns_blocking_truth


In [45]:
filename = "/data/censorship/OONI/TM_2021_06_20_to_2022_02_09groundtruth_combined.csv"
df = pd.read_csv(filename)
df_ = df[df["dns_blocking_truth"]=="Confirmed"]



,Unnamed: 0,input,measurement_start_time,probe_asn,probe_cc,probe_ip,resolver_asn,test_keys_ipv4,dns_blocking_truth
48,48,https://hootsuite.com/,2021-07-10 08:21:04,AS51495,TM,127.0.0.1,AS0,['127.0.0.1'],Confirmed
57,57,https://fri-gate.org/ru/,2021-07-10 08:23:15,AS51495,TM,127.0.0.1,AS0,['127.0.0.1'],Confirmed
77,77,https://meduza.io/,2021-07-10 08:25:52,AS51495,TM,127.0.0.1,AS0,['127.0.0.1'],Confirmed
92,92,https://openvpn.net/,2021-07-10 08:28:21,AS51495,TM,127.0.0.1,AS0,['127.0.0.1'],Confirmed
103,103,http://www.rferl.org/,2021-07-10 08:30:30,AS51495,TM,127.0.0.1,AS0,['127.0.0.1'],Confirmed
...,...,...,...,...,...,...,...,...,...
2569,2569,https://scontent.xx.fbcdn.net/robots.txt,2022-02-09 18:02:15,AS20661,TM,127.0.0.1,AS15169,['127.0.0.1'],Confirmed
2570,2570,https://fbcdn.net/robots.txt,2022-02-09 18:02:15,AS20661,TM,127.0.0.1,AS15169,['127.0.0.1'],Confirmed
2577,2577,https://pbs.twimg.com/profile_images/941658150...,2022-02-09 18:02:41,AS20661,TM,127.0.0.1,AS15169,['127.0.0.1'],Confirmed
2581,2581,http://www.linkedin.com/,2022-02-09 18:02:46,AS20661,TM,127.0.0.1,AS15169,['127.0.0.1'],Confirmed
